In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer

PLOTS_DIR = f"plots/regression_transformer_pil/"
MODEL_DIR = f"models/regression_transformer_pil/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

2026-01-29 13:01:02.138166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769688062.160803 1842930 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769688062.168571 1842930 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769688062.186929 1842930 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769688062.186950 1842930 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769688062.186953 1842930 computation_placer.cc:177] computation placer alr

In [2]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatFullReconstructor(data_config)

In [3]:
Transformer.build_model(
    hidden_dim=128,
    num_layers=6,
    dropout_rate=0.1,
)


I0000 00:00:1769688173.492583 1842930 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14089 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [4]:
Transformer.adapt_normalization_layers(X_train)
Transformer.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
        "normalized_regression": core.utils.RegressionLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy(name="assignment_accuracy")],
        "normalized_regression": [core.utils.RegressionDeviation()],
    },
    add_physics_informed_loss=True,
    #loss_weights={"physics_informed_loss": 0.1, "assignment": 1.0, "normalized_regression": 1.0}
)
Transformer.model.summary()

I0000 00:00:1769688176.715661 1844781 service.cc:152] XLA service 0x7ff09c001dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769688176.715688 1844781 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-01-29 13:02:56.726587: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1769688176.742481 1844781 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1769688176.879614 1844781 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization
Adapted normalization layer:  met_input_normalization
Set regression layer stats.
Added physics-informed loss to the model.
Compiling model with physics informed loss. Ensure that the loss dictionary includes 'reco_mass_deviation'.


Model: "keras_model_wrapper_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ConcatLeptonCharge  │ (None, 2, 4)      │          0 │ lep_input_normal… │
│ (ConcatLeptonCharg… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_mlp   │ (None, 6, 128)    │      9,118 │ jet_input_normal… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_m… │ (None, 2, 128)    │      8,396 │ ConcatLeptonChar… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_embedding_mlp   │ (None, 1, 128)    │      6,764 │ met_input_normal… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 9, 128)    │          0 │ jet_embedding_ml… │
│ (Concatenate)       │                   │            │ lepton_embedding… │
│                     │                   │            │ met_embedding_ml… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_jet_mask     │ (None, 9)         │          0 │ jet_mask[0][0]    │
│ (ExpandJetMask)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,196,431 (4.56 MB)

 Trainable params: 1,196,394 (4.56 MB)

 Non-trainable params: 37 (160.00 B)

In [ ]:
Transformer.train_model(
    epochs=15,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
)

Prepared regression targets with normalization.
Epoch 1/15


2026-01-29 13:04:00.265487: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_slice_fusion_1', 44 bytes spill stores, 44 bytes spill loads



3121/3122 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - assignment_assignment_accuracy: 0.2842 - assignment_loss: 0.1385 - loss: 1.2744 - normalized_regression_deviation: 0.5233 - normalized_regression_loss: 0.9273 - reco_mass_deviation_loss: 0.2085

2026-01-29 13:07:07.162962: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_3', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_4', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion', 8 bytes spill stores, 8 bytes spill loads



3122/3122 ━━━━━━━━━━━━━━━━━━━━ 251s 64ms/step - assignment_assignment_accuracy: 0.2843 - assignment_loss: 0.1385 - loss: 1.2743 - normalized_regression_deviation: 0.5233 - normalized_regression_loss: 0.9273 - reco_mass_deviation_loss: 0.2085 - val_assignment_assignment_accuracy: 0.6402 - val_assignment_loss: 0.0901 - val_loss: 0.8837 - val_normalized_regression_deviation: 0.4589 - val_normalized_regression_loss: 0.7020 - val_reco_mass_deviation_loss: 0.0915 - learning_rate: 1.0000e-04
Epoch 2/15
3122/3122 ━━━━━━━━━━━━━━━━━━━━ 169s 54ms/step - assignment_assignment_accuracy: 0.5608 - assignment_loss: 0.1033 - loss: 1.0032 - normalized_regression_deviation: 0.4911 - normalized_regression_loss: 0.7560 - reco_mass_deviation_loss: 0.1439 - val_assignment_assignment_accuracy: 0.6226 - val_assignment_loss: 0.0915 - val_loss: 1.0075 - val_normalized_regression_deviation: 0.4650 - val_normalized_regression_loss: 0.7364 - val_reco_mass_deviation_loss: 0.1795 - learning_rate: 1.0000e-04
Epoch 3/1

In [ ]:
Transformer.save_model(f"{MODEL_DIR}/model.keras")

In [ ]:
from core.components import reco_W_mass_deviation

import tensorflow as tf
import numpy as np

In [ ]:
reco_W_mass_deviation(
    lepton_momenta=tf.cast(X_train["lep_inputs"], tf.float32),
    neutrino_momenta=tf.cast(X_train["neutrino_truth"], tf.float32),
).numpy().mean()

In [ ]:
reco_W_mass_deviation(
    lepton_momenta=tf.cast(X_train["lep_inputs"], tf.float32),
    neutrino_momenta=tf.cast(X_train["nu_flows_neutrino_truth"], tf.float32),
).numpy().mean()

In [ ]:
neutrino_pred = Transformer.reconstruct_neutrinos({"jet": X_train["jet_inputs"], "lepton": X_train["lep_inputs"], "met": X_train["met_inputs"]})
reco_W_mass_deviation(
    lepton_momenta=tf.cast(X_train["lep_inputs"], tf.float32),
    neutrino_momenta=tf.cast(neutrino_pred, tf.float32),
).numpy().mean()

In [ ]:
np.mean(neutrino_pred, axis= 0)

In [ ]:
upscale_layer = Transformer.model.get_layer("regression")

In [ ]:
upscale_layer.weights

In [ ]:
np.std(X_train["neutrino_truth"], axis=0)